# Installation

In [ ]:
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git
!apt-get install poppler-utils 

     |████████████████████████████████| 2.5 MB 5.2 MB/s 
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-54ol6gaf
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-54ol6gaf
     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 14.1 MB 30 kB/s 
     |████████████████████████████████| 2.5 MB 35.0 MB/s 
     |████████████████████████████████| 51 kB 18 kB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 321 kB 43.6 MB/s 
     |████████████████████████████████| 348 kB 36.8 MB/s 
     |████████████████████████████████| 85 kB 4.1 MB/s 
     |████████████████████████████████| 981 kB 37.6 MB/s 
     |████████████████████████████████| 3.0 MB 36.0 MB/s 
     |████████████████████████████████| 78 kB 6.1 MB/s 
     |████████████████████████████████| 5.6 MB 34.7 MB/s 
     |███████████

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 1s (251 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.document_store.memory import InMemoryDocumentStore
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers
from haystack.preprocessor.preprocessor import PreProcessor
from haystack.file_converter.pdf import PDFToTextConverter


/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


# Initialise DocStore

In [ ]:
# from haystack.document_store import FAISSDocumentStore

# document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

In [ ]:
# document_store = InMemoryDocumentStore()

In [ ]:
# ####### for embeddingretrieval ###


# document_store = InMemoryDocumentStore(similarity="cosine",embedding_dim=4096)

In [ ]:
######## Clear doc store  #############
InMemoryDocumentStore.delete_all_documents
document_store = InMemoryDocumentStore(similarity="dot_product")

# Storing in DocStore by clean_wiki_text


In [ ]:
#########   METHOD 1 ############
# Let's first get some files that we want to use
doc_dir = "/content/x"
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

In [ ]:
xx = len(dicts)
print(dicts)

[{'content': 'We wanted to find a way we could engage\nwith our people and you to help bring our\nThat\'s why we created this short video\nthat demonstrates how `People\' create\nvalue at the point of delivery.\nwww.vimeo.com/417101268/b5b5a8faad\nWe will deliver a step change in\nlooked after across all corners\nThis will ensure our people are\nrespected, listened to, guided,\nyou. We\'ll keep up to date with\nrewarding people for a good job,\ninitiatives to recognise the team,\nany barriers between silos, and\nparty supplier feeling to a more\n\x0c6.4  Contract management structure\nWe\'ll introduce our People Programme to develop, respect and nurture our teams\nWe know there is a good team currently in place\nand, as the existing people will transfer to us, we will\nembrace and celebrate the dedication and quality\nof the team through the introduction of our People\nOur People Programme covers all aspects of the\nemployee lifecycle, from the initial workshop with you\nto design the 

# Storing in DocStore by Converter method

In [ ]:
######### METHOD 2 ########
####### text to doc ######
#########
u = '/content/2. People - Salisbury Group.pdf'
converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_pdf = converter.convert(file_path=u)


#document_store = InMemoryDocumentStore()
pp = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
    split_overlap=3
)
docs = pp.process(doc_pdf)


document_store.write_documents(docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


One or more sentence found with word count higher than the split length.


In [ ]:
import os
#document_store.write_documents(docs)
directory = '/content'
for filename in os.listdir(directory):
    if filename.endswith(".pdf") or filename.endswith(".doc"):
        converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])
        doc_pdf = converter.convert(directory+'/'+filename)


        #document_store = InMemoryDocumentStore()
        pp = PreProcessor(
            clean_empty_lines=True,
            clean_whitespace=True,
            clean_header_footer=True,
            split_by="word",
            split_length=100,
            split_respect_sentence_boundary=True,
            split_overlap=3
        )
        docs = pp.process(doc_pdf)
        document_store.write_documents(docs)
    else:
        print("Exit")


Exit


One or more sentence found with word count higher than the split length.
One or more sentence found with word count higher than the split length.


Exit


# Sentence-Based Splitting

In [ ]:
######### METHOD 3 ########
####### text to doc ######
#########

converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_pdf = converter.convert(file_path="/content/x/2. People - Salisbury Group.pdf")


#document_store = InMemoryDocumentStore()
pp1 = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="sentence",
    split_length=10,
    split_respect_sentence_boundary=False,
    split_overlap=3
)
docs = pp1.process(doc_pdf)


document_store.write_documents(docs)

# Retriever

In [ ]:
######### DPR #########
from haystack.retriever.dense import DensePassageRetriever
retriever_DPR = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=True,
                                  embed_title=True,
                                  use_fast_tokenizers=True)
# Important: 
# Now that after we have the DPR initialized, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation. 
# While this can be a time consuming operation (depending on corpus size), it only needs to be done once. 
# At query time, we only need to embed the query and compare it the existing doc embeddings which is very fast.
document_store.update_embeddings(retriever_DPR)

Updating Embedding:   0%|          | 0/38 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/48 [00:00<?, ? Docs/s]

Documents Processed: 10000 docs [00:01, 6851.53 docs/s]


In [ ]:
#########  TFIDF  ###########
# An in-memory TfidfRetriever based on Pandas dataframes

from haystack.retriever.sparse import TfidfRetriever
retriever_TF = TfidfRetriever(document_store=document_store)

In [ ]:
########## EmbeddingRetriever #########
from haystack.retriever import EmbeddingRetriever
retriever_emb = EmbeddingRetriever(document_store=document_store,
                               embedding_model="ahotrod/albert_xxlargev1_squad2_512")
document_store.update_embeddings(retriever_emb)

Some weights of the model checkpoint at ahotrod/albert_xxlargev1_squad2_512 were not used when initializing AlbertModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Currently no support in Processor for returning problematic ids
ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.53s/ Batches]
Documents Processed: 10000 docs [00:02, 4075.38 docs/s]


# Reader

In [ ]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

#reader = FARMReader(model_name_or_path="deepset/roberta-large-squad2", use_gpu=True)
reader = FARMReader(model_name_or_path="ahotrod/albert_xxlargev1_squad2_512", use_gpu=True) #######  best in accuracy
#reader1 = TransformersReader(model_name_or_path="ahotrod/albert_xxlargev1_squad2_512")
#reader = FARMReader(model_name_or_path="deepset/bert-large-uncased-whole-word-masking-squad2", use_gpu=True)
#reader = FARMReader(model_name_or_path="flozi00/albert-base-v2-squad_v2", use_gpu=True)


Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/890M [00:00<?, ?B/s]

Some weights of the model checkpoint at ahotrod/albert_xxlargev1_squad2_512 were not used when initializing AlbertModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.


In [ ]:
# reader2 = FARMReader(model_name_or_path="ahotrod/albert_xxlargev1_squad2_512", 
#               context_window_size=50, no_ans_boost=0, batch_size=200,
#               use_gpu=True)

In [ ]:
# #del reader2
# torch.cuda.empty_cache()

# Pipeline

In [ ]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever_TF)

# Query

In [ ]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k for retriever, the better (but also the slower) your answers.

#How are you different than other FM companies?
#How do you handle challenges?
#How do you innovate?
#How do you manage performance?

#Describe people programme
#What are the responsibilities of security officer?
#how will you improve our flow?
#Will your Account Manager will take ownership and responsibility for all services?

#How do you automate production of reports?
#How do you produce reports?
#Employee savings


prediction = pipe.run(
    query="How are you different than other FM companies?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.23 Batches/s]


In [ ]:
prediction = pipe.run(
    query="What are the responsibilities of security officer?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.13 Batches/s]


In [ ]:
prediction = pipe.run(
    query="How are you different than other FM companies?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.14 Batches/s]


# Result

In [ ]:
print_answers(prediction)

{   'answers': [   Answer(answer='delivery of site-based security operations', type='extractive', score=0.4398578852415085, context='Security Officer\nJob Profile\nEffective delivery of site-based security operations\nacting in the best interests of our clients and\ncustomers always, en', offsets_in_document=[Span(start=38, end=80)], offsets_in_context=[Span(start=38, end=80)], document_id='a73008dd01d0701ef94614162380e6ec', meta={'_split_id': 180}),
                   Answer(answer='Reception Didcot x 1', type='extractive', score=0.09290205128490925, context='Security Officer/\nReception Didcot x 1', offsets_in_document=[Span(start=17, end=37)], offsets_in_context=[Span(start=17, end=37)], document_id='bdee782941f657019e5d156bdfec756', meta={'_split_id': 129}),
                   Answer(answer='Reception Didcot x 1', type='extractive', score=0.09290205128490925, context='Security Officer/\nReception Didcot x 1', offsets_in_document=[Span(start=17, end=37)], offsets_in_context=[Span(st

# Extras

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer

model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")
tokenizer = BertTokenizer.from_pretrained('tokenizer_tf2_qa')

questions = ['How are you different than other FM companies?',
             'How do you handle challenges?',
             'How do you innovate?',
             'How do you manage performance?']
paragraph = dicts[0]

for question in questions:
  question_tokens = tokenizer.tokenize(question)
  paragraph_tokens = tokenizer.tokenize(paragraph)
  tokens = ['[CLS]'] + question_tokens + ['[SEP]'] + paragraph_tokens + ['[SEP]']
  input_word_ids = tokenizer.convert_tokens_to_ids(tokens)
  input_mask = [1] * len(input_word_ids)
  input_type_ids = [0] * (1 + len(question_tokens) + 1) + [1] * (len(paragraph_tokens) + 1)
  input_word_ids, input_mask, input_type_ids = map(lambda t: tf.expand_dims(
      tf.convert_to_tensor(t, dtype=tf.int32), 0), (input_word_ids, input_mask, input_type_ids))
  outputs = model([input_word_ids, input_mask, input_type_ids])
  short_start = tf.argmax(outputs[0][0][1:]) + 1
  short_end = tf.argmax(outputs[1][0][1:]) + 1
  answer_tokens = tokens[short_start: short_end + 1]
  answer = tokenizer.convert_tokens_to_string(answer_tokens)
  print(f'Question: {question}')
  print(f'Answer: {answer}')



OSError: ignored

In [ ]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.generator.transformers import Seq2SeqGenerator

In [ ]:
from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore(vector_dim=128, faiss_index_factory_str="Flat")

In [ ]:
doc_dir = "/content/x"
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)
document_store.write_documents(dicts)

In [ ]:
from haystack.retriever.dense import EmbeddingRetriever

retriever = EmbeddingRetriever(document_store=document_store,
                               embedding_model="yjernite/retribert-base-uncased",
                               model_format="retribert")

document_store.update_embeddings(retriever)

Some weights of RetriBertModel were not initialized from the model checkpoint at yjernite/retribert-base-uncased and are newly initialized: ['bert_query.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Documents Processed: 10000 docs [00:00, 96686.17 docs/s]


In [ ]:
generator = Seq2SeqGenerator(model_name_or_path="yjernite/bart_eli5")

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:
from haystack.pipeline import GenerativeQAPipeline
pipe = GenerativeQAPipeline(generator, retriever)

In [ ]:
prediction = pipe.run(
    query="How are you different than other FM companies?", params={"Retriever": {"top_k": 5}}
)
print_answers(prediction)

{   'answers': [   ' You are different from other FM companies in the same way '
                   'that you are different than other Fortune 500 companies.'],
    'documents': [   {'text': "Services for a sustainable life\nWe are pleased to submit our proposal\nfor this exciting opportunity. All of us\nat Salisbury have all been enthused\nby your team's plans and approach\nand we believe that a true alignment\nexists between both our organisations'\ncommitments to people and a quality\nWe understand that you are trying to shape an FM\nsolution that supports RWE's goals and builds upon\nthe investment in your business and estate. You are\nlooking for a partner that lives and breathes this\ncommitment and can work with you to improve service\ndelivery in a sustainable way over the long term.\nPartnership relationships are in our DNA: we were\nlaunched following a client insourcing process in 2012\nand because of this, we understand what it takes to\nalign interests, provide transparenc